# Load packages

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

# Set up paths

In [ ]:
code_dir = Path.cwd()
statistics_dir = code_dir.parent
source_dir = statistics_dir / "input"
output_dir = statistics_dir / "output/linear_models"
output_dir.mkdir(exist_ok=True, parents=True)

# Load data

In [ ]:
statistics_df = pd.read_csv(source_dir / 'statistics_df_randomized.csv', index_col = "sub_id")

# Set up R environment

In [ ]:

import os
import rpy2.robjects as robjects

# Set the R_HOME environment variable
os.environ['R_HOME'] = '/usr/lib/R/'

# Update the library paths
new_path = "/home/csi/R/x86_64-pc-linux-gnu-library/4.3"
robjects.r(f'.libPaths(c("{new_path}", .libPaths()))')


# Linear mixed effects models comparing BBB leakage between lesion, penumbra and normal tissue

In [ ]:
from pymer4.models import Lmer

# Prepare dataframe
repeatead_measures = ['nice_normal_z_ef','nice_penumbra_z_ef','nice_lesion_z_ef']
df_lme = pd.melt(statistics_df.reset_index(), 
                 id_vars=['sub_id','AGE', 'SEX', 'NIHSSSCORE_V00', 'stroke_volume_v00', 'scanner'], 
                 value_vars=repeatead_measures, 
                 var_name="loc_measurement", 
                 value_name="EF", ignore_index=False).reset_index(drop=True)

# Z-score age, stroke volume, NIHSS and EF
columns_to_normalize = ['AGE', 'stroke_volume_v00', 'NIHSSSCORE_V00', 'EF']
df_lme[columns_to_normalize] = df_lme[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())

# Initialize model instance using 1 predictor with random intercepts and slopes
model = Lmer("EF ~ loc_measurement + AGE + SEX + NIHSSSCORE_V00 + stroke_volume_v00 + (1|sub_id) + (1|scanner)", data=df_lme)

# Fit LMM 
lme = model.fit(factors={"loc_measurement": ['nice_normal_z_ef','nice_lesion_z_ef', 'nice_penumbra_z_ef']})
lme.to_csv(output_dir / "lesion_penumbra_normal_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "lesion_penumbra_normal_anova.csv")
anova

In [ ]:
# Compute post-hoc tests
marginal_estimates, comparisons = model.post_hoc(marginal_vars="loc_measurement", grouping_vars="loc_measurement")

# "Cell" means of the ANOVA
comparisons.to_csv(output_dir/"lesion_penumbra_normal_posthoc.csv")
print(comparisons)


In [ ]:
repeatead_measures = ['nice_penumbra_noinfarct_z_ef', 'nice_penumbra_infarct_z_ef']
statistics_df[repeatead_measures].describe()

## Visualization

In [ ]:
location_styled = [
    'Normal Tissue',
    'Penumbra (Tmax>6s)',
    'Infarct Core'
]

ef_styled = ['EF (z-scored)']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

repeatead_measures = ['nice_normal_z_ef','nice_penumbra_z_ef','nice_lesion_z_ef']
df_lme = pd.melt(statistics_df.reset_index(), 
                 id_vars=['sub_id','AGE', 'SEX', 'NIHSSSCORE_V00', 'stroke_volume_v00', 'scanner'], 
                 value_vars=repeatead_measures, 
                 var_name="loc_measurement", 
                 value_name="EF", ignore_index=False).reset_index(drop=True)

custom_palette = sns.color_palette("Paired")


plt.figure(figsize=(8, 6))
sns.boxplot(y='EF', x='loc_measurement', data=df_lme, width=0.5, palette=custom_palette, boxprops=dict(alpha=0.7), showfliers=False)
sns.stripplot(y='EF', x='loc_measurement', data=df_lme, color='black', size=3, jitter=True, alpha=0.6)

# Annotations for significance
y_max = df_lme['EF'].max()
y_step = y_max * 0.12  # Calculate step size for annotations based on the maximum EF value

# Ensure the calculation of 'y' is correct by explicitly converting 'i' and 'y_step' to compatible types
for i, row in comparisons.iterrows():
    groups = row['Contrast'].split(' - ')
    p_value = row['P-val']
    
    # Assuming group names in 'loc_measurement' match those in 'Comparison'
    group_labels = df_lme['loc_measurement'].unique().tolist()
    x1 = group_labels.index(groups[0])
    x2 = group_labels.index(groups[1])
    
    # Correctly calculate 'y' by ensuring 'i' and 'y_step' are compatible types
    y = y_max + ((float(i)) * y_step)  # Added (i+1) to ensure spacing starts above the max value
    
    # Adjust 'p_text' based on your significance criteria
    p_text = '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'
    
    plt.plot([x1, x1, x2, x2], [y - y_step/4, y, y, y - y_step/4], lw=1.5, c='black')
    plt.text((x1 + x2) * 0.5, y, p_text, ha='center', va='bottom')


plt.xticks(ticks=np.arange(len(location_styled)), labels=location_styled, size=10)
plt.xlabel('')
plt.ylabel('EF (z-scored)')

plt.savefig(output_dir/"boxplot_lesion_penumbra_normal_ef.png", dpi=300)


# Linear mixed effects models comparing BBB leakage within the perfusion deficit

In [ ]:
from pymer4.models import Lmer

# Prepare dataframe
statistics_df_clean = statistics_df.dropna(subset=['nice_penumbra_z_ef'])
repeatead_measures = ['nice_tmax6_z_ef','nice_tmax8_z_ef', 'nice_tmax10_z_ef']
df_lme = pd.melt(statistics_df_clean.reset_index(), 
                 id_vars=['sub_id','AGE', 'SEX', 'NIHSSSCORE_V00', 'stroke_volume_v00', 'scanner'], 
                 value_vars=repeatead_measures, 
                 var_name="loc_measurement", 
                 value_name="EF", ignore_index=False).reset_index(drop=True)

# Z-score age, stroke volume, NIHSS and EF
columns_to_normalize = ['AGE', 'stroke_volume_v00', 'NIHSSSCORE_V00', 'EF']
df_lme[columns_to_normalize] = df_lme[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())

# Define model
model = Lmer("EF ~ loc_measurement + AGE + SEX + NIHSSSCORE_V00 + stroke_volume_v00 + (1|sub_id) + (1|scanner)", data=df_lme)

# Fit LMM 
lme = model.fit(factors={"loc_measurement": ['nice_tmax6_z_ef','nice_tmax8_z_ef', 'nice_tmax10_z_ef']})
lme.to_csv(output_dir / "perfdef_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "perfdef_anova.csv")
anova

In [ ]:
statistics_df_clean[repeatead_measures].describe()

In [ ]:
# Compute post-hoc tests
marginal_estimates, comparisons = model.post_hoc(marginal_vars="loc_measurement", grouping_vars="loc_measurement")

# "Cell" means of the ANOVA
comparisons.to_csv(output_dir/"perfdef_posthoc.csv")
print(comparisons)


In [ ]:
location_styled = [
    'Tmax 6-8s',
    'Tmax 8-10s',
    'Tmax >/=10s'
]

ef_styled = ['EF (z-scored)']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

statistics_df_clean = statistics_df.dropna(subset=['nice_penumbra_z_ef'])
repeatead_measures = ['nice_tmax6_z_ef','nice_tmax8_z_ef', 'nice_tmax10_z_ef']
df_lme = pd.melt(statistics_df_clean.reset_index(), 
                 id_vars=['sub_id','AGE', 'SEX', 'NIHSSSCORE_V00', 'stroke_volume_v00', 'scanner'], 
                 value_vars=repeatead_measures, 
                 var_name="loc_measurement", 
                 value_name="EF", ignore_index=False).reset_index(drop=True)

custom_palette = sns.color_palette("Paired")


plt.figure(figsize=(8, 6))
sns.boxplot(y='EF', x='loc_measurement', data=df_lme, width=0.5, palette=custom_palette, boxprops=dict(alpha=0.7), showfliers=False)
sns.stripplot(y='EF', x='loc_measurement', data=df_lme, color='black', size=3, jitter=True, alpha=0.6)

# Annotations for significance
y_max = df_lme['EF'].max()
y_step = y_max * 0.12  # Calculate step size for annotations based on the maximum EF value

# Ensure the calculation of 'y' is correct by explicitly converting 'i' and 'y_step' to compatible types
for i, row in comparisons.iterrows():
    groups = row['Contrast'].split(' - ')
    p_value = row['P-val']
    
    # Assuming group names in 'loc_measurement' match those in 'Comparison'
    group_labels = df_lme['loc_measurement'].unique().tolist()
    x1 = group_labels.index(groups[0])
    x2 = group_labels.index(groups[1])
    
    # Correctly calculate 'y' by ensuring 'i' and 'y_step' are compatible types
    y = y_max + ((float(i)) * y_step)  # Added (i+1) to ensure spacing starts above the max value
    
    # Adjust 'p_text' based on your significance criteria
    p_text = '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'
    
    plt.plot([x1, x1, x2, x2], [y - y_step/4, y, y, y - y_step/4], lw=1.5, c='black')
    plt.text((x1 + x2) * 0.5, y, p_text, ha='center', va='bottom')


plt.xticks(ticks=np.arange(len(location_styled)), labels=location_styled, size=10)
plt.xlabel('')
plt.ylabel('EF (z-scored)')

plt.savefig(output_dir/"boxplot_perfdef_ef.png", dpi=300)


# Linear mixed effects models comparing BBB leakage  within penumbra according to future infarction

In [ ]:
# EF dataframe
statistics_df_clean = statistics_df.dropna(subset=['nice_penumbra_z_ef'])
repeatead_measures = ['nice_penumbra_noinfarct_z_ef', 'nice_penumbra_infarct_z_ef']
df_ef = pd.melt(statistics_df_clean.reset_index(), 
                 id_vars=['sub_id','AGE', 'SEX', 'NIHSSSCORE_V00', 'stroke_volume_v00', 'treatment', 'scanner'], 
                 value_vars=repeatead_measures, 
                 var_name="loc_measurement", 
                 value_name="EF", ignore_index=False).reset_index(drop=True)
df_ef['loc_measurement'] = df_ef['loc_measurement'].apply(lambda x: x.split('_')[2])

# Tmax dataframe
tmax = ['nice_penumbra_noinfarct_mean_tmax_rapid', 'nice_penumbra_infarct_mean_tmax_rapid']
df_tmax = pd.melt(statistics_df_clean.reset_index(), 
                 id_vars='sub_id',
                    value_vars=tmax,
                    var_name='loc_measurement',
                    value_name='Tmax', ignore_index=False).reset_index(drop=True)

# Rename values of loc_measurement by splitting the string
df_tmax['loc_measurement'] = df_tmax['loc_measurement'].apply(lambda x: x.split('_')[2])

# Merge EF and Tmax dataframes
df_lme = pd.merge(df_ef, df_tmax, on=['sub_id', 'loc_measurement'])

# Create separate dataframes for placebo and treatment groups
df_placebo = df_lme[df_lme['treatment'] == 0 ]
df_treatment = df_lme[df_lme['treatment'] == 1 ]

# Drop rows with NaN values in EF and Tmax
df_placebo = df_placebo.dropna(subset=['EF', 'Tmax'])
df_treatment = df_treatment.dropna(subset=['EF', 'Tmax'])
df_lme = df_lme.dropna(subset=['EF', 'Tmax'])

# Z-score age, stroke volume, NIHSS and EF
columns_to_normalize = ['AGE', 'stroke_volume_v00', 'NIHSSSCORE_V00', 'EF']
df_lme[columns_to_normalize] = df_lme[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())
df_placebo[columns_to_normalize] = df_placebo[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())
df_treatment[columns_to_normalize] = df_treatment[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())

## Model controlling for tmax, treatment, and the interaction of treatment with location of measurement

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("EF ~ loc_measurement + Tmax + AGE + SEX + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:loc_measurement + (1|sub_id) + (1|scanner)", data=df_lme)

# Fit LMM 
lme = model.fit(factors={"loc_measurement": ['noinfarct', 'infarct']})
lme.to_csv(output_dir / "penumbra_adjusted_interaction_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_adjusted_interaction_anova.csv")
anova

In [ ]:
# Compute post-hoc tests
marginal_estimates, comparisons = model.post_hoc(marginal_vars="loc_measurement", grouping_vars="loc_measurement")

# "Cell" means of the ANOVA
comparisons.to_csv(output_dir/"penumbra_adjusted_interaction_posthoc.csv")
print(comparisons)

## Visualization stratified by treatment status

In [ ]:
location_styled = [
    'Salvaged Penumbra', 
    'Lost Penumbra'
]

ef_styled = ['EF (z-scored)']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

statistics_df_clean = statistics_df.dropna(subset=['nice_penumbra_z_ef'])
repeatead_measures = ['nice_penumbra_noinfarct_z_ef', 'nice_penumbra_infarct_z_ef']
df_lme = pd.melt(statistics_df_clean.reset_index(), 
                 id_vars=['sub_id','AGE', 'SEX', 'NIHSSSCORE_V00', 'stroke_volume_v00', 'scanner', 'treatment'], 
                 value_vars=repeatead_measures, 
                 var_name="loc_measurement", 
                 value_name="EF", ignore_index=False).reset_index(drop=True)

df_lme['loc_measurement'] = df_lme['loc_measurement'].apply(lambda x: x.split('_')[2])

custom_palette = sns.color_palette("Paired")

plt.figure(figsize=(8, 6))
sns.stripplot(y='EF', x='loc_measurement', hue='treatment', data=df_lme, dodge=True, legend=False, color='black', size=3, jitter=True, alpha=0.6)
sns.boxplot(y='EF', x='loc_measurement', hue='treatment', data=df_lme, dodge=True, gap=0.2, legend=False, width=0.3, palette=custom_palette, boxprops=dict(alpha=0.7), showfliers=False)

# Annotations for significance
y_max = df_lme['EF'].max()
y_step = y_max * 0.12  # Calculate step size for annotations based on the maximum EF value

# Ensure the calculation of 'y' is correct by explicitly converting 'i' and 'y_step' to compatible types
for i, row in comparisons.iterrows():
    groups = row['Contrast'].split(' - ')
    p_value = row['P-val']
    
    # Assuming group names in 'loc_measurement' match those in 'Comparison'
    group_labels = df_lme['loc_measurement'].unique().tolist()
    x1 = group_labels.index(groups[0])
    x2 = group_labels.index(groups[1])
    
    # Correctly calculate 'y' by ensuring 'i' and 'y_step' are compatible types
    y = y_max + ((float(i)) * y_step)  # Added (i+1) to ensure spacing starts above the max value
    
    # Adjust 'p_text' based on your significance criteria
    p_text = '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'
    
    plt.plot([x1, x1, x2, x2], [y - y_step/4, y, y, y - y_step/4], lw=1.5, c='black')
    plt.text((x1 + x2) * 0.5, y, p_text, ha='center', va='bottom')


plt.xticks(ticks=np.arange(len(location_styled)), labels=location_styled, size=10)
plt.xlabel('')
plt.ylabel('EF (z-scored)')


plt.savefig(output_dir/"boxplot_penumbra_interaction_adjusted_ef.png", dpi=300)


## Post-hoc LMEs for placebo and treatment group separately 

### PLACEBO

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("EF ~ loc_measurement + Tmax + AGE + SEX + NIHSSSCORE_V00 + stroke_volume_v00 + (1|sub_id) + (1|scanner)", data=df_placebo)

# Fit LMM 
lme = model.fit(factors={"loc_measurement": ['noinfarct', 'infarct']})
lme.to_csv(output_dir / "penumbra_adjusted_placebo_interaction_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_adjusted_placebo_interaction_anova.csv")
anova

In [ ]:
# Compute post-hoc tests
marginal_estimates, comparisons = model.post_hoc(marginal_vars="loc_measurement", grouping_vars="loc_measurement")

# "Cell" means of the ANOVA
comparisons.to_csv(output_dir/"penumbra_adjusted_placebo_interaction_posthoc.csv")
print(comparisons)

### TREATMENT

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("EF ~ loc_measurement + Tmax + AGE + SEX + NIHSSSCORE_V00 + stroke_volume_v00 + (1|sub_id) + (1|scanner)", data=df_treatment)

# Fit LMM 
lme = model.fit(factors={"loc_measurement": ['noinfarct', 'infarct']})
lme.to_csv(output_dir / "penumbra_adjusted_treatment_interaction_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_adjusted_treatment_interaction_anova.csv")
anova

In [ ]:
# Compute post-hoc tests
marginal_estimates, comparisons = model.post_hoc(marginal_vars="loc_measurement", grouping_vars="loc_measurement")

# "Cell" means of the ANOVA
comparisons.to_csv(output_dir/"penumbra_adjusted_treatment_interaction_posthoc.csv")
print(comparisons)


# Future-infarction Penumbra: Voxel-wise analysis

In [ ]:
# load dataframe
fip_df = pd.read_csv(source_dir / 'statistics_df_futureinfarction.csv')

In [ ]:
# Z-score age, stroke volume, NIHSS and EF
columns_to_normalize = ['AGE', 'stroke_volume_v00', 'NIHSSSCORE_V00', 'ef_value', 'tmax_value']
fip_df[columns_to_normalize] = fip_df[columns_to_normalize].apply(lambda x: (x - x.mean()) / x.std())

## Infarct ~ EF

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("infarct ~ ef_value + AGE + SEX + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:ef_value + (ef_value|sub_id) + (ef_value|scanner)", data=fip_df, family="binomial")

# Fit LMM 
lme = model.fit(fold_optimizer=True)
lme.to_csv(output_dir / "penumbra_voxelwise_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_anova.csv")
anova

# Save fitted values
fip_df['infarct_fitted_ef'] = model.fits

## Infarct ~ EF + Tmax 

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("infarct ~ ef_value + tmax_value + AGE + SEX + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:ef_value + treatment:tmax_value + (ef_value + tmax_value|sub_id) + (ef_value + tmax_value|scanner)", data=fip_df, family="binomial")

# Fit LMM 
lme = model.fit(fold_optimizer=True)
lme.to_csv(output_dir / "penumbra_voxelwise_ef_tmax_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_ef_tmax_anova.csv")
anova

# Save fitted values
fip_df['infarct_fitted_ef_tmax'] = model.fits

## Infarct ~ Tmax

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("infarct ~ tmax_value + AGE + SEX + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:tmax_value + (tmax_value|sub_id) + (tmax_value|scanner)", data=fip_df, family="binomial")

# Fit LMM 
lme = model.fit(fold_optimizer=True)
lme.to_csv(output_dir / "penumbra_voxelwise_tmax_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_tmax_anova.csv")
anova

# Save fitted values
fip_df['infarct_fitted_tmax'] = model.fits

## EF ~ Tmax

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("ef_value ~ tmax_value + AGE + SEX + NIHSSSCORE_V00 + stroke_volume_v00 + (tmax_value|sub_id) + (tmax_value|scanner)", data=fip_df)

# Fit LMM 
lme = model.fit(fold_optimizer=True)
lme.to_csv(output_dir / "penumbra_voxelwise_tmax_ef_corr_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "penumbra_voxelwise_tmax_ef_corr_anova.csv")
anova

In [ ]:
fip_df.to_csv(output_dir / "statistics_df_futureinfarction_fitted.csv", index=False)

In [ ]:
fip_df = pd.read_csv(output_dir / "statistics_df_futureinfarction_fitted.csv")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.utils import resample
import statsmodels.stats.api as sms

fitted_value_dictionary = {
    'EF': 'infarct_fitted_ef', 
    'Tmax': 'infarct_fitted_tmax', 
    'EF & Tmax': 'infarct_fitted_ef_tmax'
}

results_dictionary = {}

# Function to calculate AUC using bootstrap sampling
def bootstrap_auc(data, fitted_col, n_iterations=1000, alpha=0.05):
    aucs = []
    for i in range(n_iterations):
        # Bootstrap sample
        sample = resample(data)
        # Calculate AUC
        aucs.append(roc_auc_score(sample['infarct'], sample[fitted_col]))
    # Calculate confidence intervals
    lower_bound = sms.DescrStatsW(aucs).tconfint_mean(alpha=alpha)[0]
    upper_bound = sms.DescrStatsW(aucs).tconfint_mean(alpha=alpha)[1]
    # lower_bound = np.percentile(aucs, (1 - alpha) / 2 * 100)
    # upper_bound = np.percentile(aucs, (1 + alpha) / 2 * 100)
    return np.mean(aucs), lower_bound, upper_bound

plt.figure()

# Loop through each fitted value column
for image_variable, fitted_col in fitted_value_dictionary.items():
    # Compute ROC Curve and AUC
    fpr, tpr, thresholds = roc_curve(fip_df['infarct'], fip_df[fitted_col])
    roc_auc = auc(fpr, tpr)

    # Calculate confidence intervals for AUC
    auc_mean, ci_lower, ci_upper = bootstrap_auc(fip_df, fitted_col)
    results_dictionary[image_variable] = (roc_auc, ci_lower, ci_upper)
    
    # Plot ROC curve for the current fitted model
    plt.plot(fpr, tpr, label=f'{image_variable}: AUC = {roc_auc:.3f}')

# Plot the diagonal line for random predictions
plt.plot([0, 1], [0, 1], color='red', linestyle='--')

# Set the limits and labels
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')

# Save the plot
plt.savefig(output_dir / 'roc_curve.png')

# Models with clinical outcomes

## Set up dataframe

In [ ]:
from scipy.stats import zscore

# Calculate change in stroke volume from V00 to V03 and binarize it
statistics_df["stroke_volume_delta"] = (statistics_df['stroke_volume_v03'] - statistics_df['stroke_volume_v00'])
statistics_df["quant_lesion_growth"] = np.where(statistics_df['stroke_volume_delta'] > 0, 1, np.where(statistics_df['stroke_volume_delta'] <= 0, 0, np.nan))

clinical_variables = [
    'AGE', 
    'SEX', 
    'NIHSSSCORE_V00', 
    'LVO_V0',
    'LVO_V3',
    'stroke_volume_v00',
    'stroke_volume_v03',
    'lesion_volume_delta',
    'quant_lesion_growth',
    'treatment', 
    'scanner',
    'SABCRIT2', 
    'neurological_sae',
    'excellent_outcome',
    'good_outcome',
    'NIHSSSCORE_V05',
    'MRSSCORE_V05',
    'rapid_pwi_volume_v03_binary' 
]

imaging_variables = [
    'nice_normal_z_ef', 
    'nice_penumbra_z_ef',
    'nice_lesion_z_ef',
    'nice_focal_normal_z_ef',
    'nice_focal_penumbra_z_ef',
    'nice_focal_lesion_z_ef'
]

continuous_variables = imaging_variables + ['AGE', 'NIHSSSCORE_V00', 'NIHSSSCORE_V05', 'stroke_volume_v00', 'lesion_volume_delta']

# Create df with clinical and imaging variables
df_clinical = statistics_df[clinical_variables + imaging_variables]
df_clinical_penumbra = df_clinical.dropna(subset=['nice_penumbra_z_ef'])

# Z-score dataframes
for col in continuous_variables:
    df_clinical[col] = zscore(df_clinical[col], nan_policy="omit")
    df_clinical_penumbra[col] = zscore(df_clinical_penumbra[col], nan_policy="omit")

## Logistic regression model with random effects: EF (lesion) ~ hemorrhage

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("SABCRIT2 ~ nice_lesion_z_ef + AGE + SEX + stroke_volume_v00 + NIHSSSCORE_V00 + treatment + treatment:nice_lesion_z_ef + (nice_lesion_z_ef|scanner)", data=df_clinical, family='binomial')

# Fit LMM 
lme = model.fit(old_optimizer=True)
lme.to_csv(output_dir / "hemorrhage_ef_lesion_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "hemorrhage_ef_lesion_anova.csv")
anova

## Logistic regression model with random effects: EF (penumbra) ~ hemorrhage

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("SABCRIT2 ~ nice_penumbra_z_ef + AGE + SEX + stroke_volume_v00 + NIHSSSCORE_V00 + treatment + treatment:nice_penumbra_z_ef + (nice_penumbra_z_ef|scanner)", data=df_clinical_penumbra, family='binomial')

# Fit LMM 
lme = model.fit(old_optimizer=True)
lme.to_csv(output_dir / "hemorrhage_ef_penumbra_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "hemorrhage_ef_penumbra_anova.csv")
anova

## Linear mixed effects model: EF (penumbra) ~ Excellent outcome (mRS 90 days 0-1)

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("excellent_outcome ~ nice_penumbra_z_ef + AGE + SEX + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:nice_penumbra_z_ef + (nice_penumbra_z_ef|scanner)", data=df_clinical_penumbra, family='binomial')

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "excellent_ef_penumbra_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "excellent_ef_penumbra_anova.csv")
anova

## Linear mixed effects model: EF (lesion) ~ Excellent outcome (mRS 90 days 0-1)

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("excellent_outcome ~ nice_lesion_z_ef + AGE + SEX + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:nice_lesion_z_ef + (nice_lesion_z_ef|scanner)", data=df_clinical, family='binomial')

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "excellent_ef_lesion_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "excellent_ef_lesion_anova.csv")
anova

## Linear mixed effects model: EF (penumbra) ~ NIHSS at 90 days

In [ ]:
from pymer4.models import Lmer

# Define model
model = Lmer("NIHSSSCORE_V05 ~ nice_penumbra_z_ef + AGE + SEX + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:nice_penumbra_z_ef + (nice_penumbra_z_ef|scanner)", data=df_clinical_penumbra)

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "nihss_ef_penumbra_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "nihss_ef_penumbra_anova.csv")
anova


## Linear mixed effects model: EF (lesion) ~ NIHSS at 90 days

In [ ]:
from pymer4.models import Lmer

# Define model
df = df_clinical.dropna(subset=['nice_lesion_z_ef']) 
model = Lmer("NIHSSSCORE_V05 ~ nice_lesion_z_ef + AGE + SEX + NIHSSSCORE_V00 + stroke_volume_v00 + treatment + treatment:nice_lesion_z_ef + (nice_lesion_z_ef|scanner)", data=df)

# Fit LMM 
lme = model.fit()
lme.to_csv(output_dir / "nihss_ef_lesion_lme.csv")
print(lme)

# Get ANOVA table
anova = model.anova()
anova.to_csv(output_dir / "nihss_ef_lesion_anova.csv")
anova